In [3]:
import pandas as pd
import numpy as np
df_Jan = pd.read_parquet(path="Data/yellow_tripdata_2023-01.parquet")
df_Feb = pd.read_parquet(path="Data/yellow_tripdata_2023-02.parquet")
pd.options.mode.chained_assignment = None 


### Question 1

In [4]:
print(f"There are {df_Jan.shape[1]} columns in this dataset.")

There are 19 columns in this dataset.


### Question 2

In [5]:
def get_duration(row : pd.Series) -> float:
    """Returns the duration between dropoff and pickup times

    Args:
        row (pd.Series): Slice of dataframe containing "tpep_pickup_datetime" and "tpep_dropoff_datetime"

    Returns:
        float: The duration of the trip
    """
    
    duration = row["tpep_dropoff_datetime"] - row["tpep_pickup_datetime"]
    minutes = duration.total_seconds() / 60.0
    return minutes

In [6]:
df_Jan["duration"] = df_Jan.apply(get_duration, axis = 1)
cov = np.cov(df_Jan.duration, bias = False) # We use (N - 1) because we want to calculate sample covariance
print(f"The standard deviation of the trips in January is {np.sqrt(cov)}.")

The standard deviation of the trips in January is 42.59435124195483.


### Question 3

In [7]:
rows = (df_Jan.duration >= 1) & (df_Jan.duration <= 60)
print(f"{rows.sum() / df_Jan.shape[0] * 100}% of the data is left")
df_Jan_subset = df_Jan.loc[rows, :]

98.1220282212598% of the data is left


In [8]:
df_Jan_subset

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,duration
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.30,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00,8.433333
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.90,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00,6.316667
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.90,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00,12.750000
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.10,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25,9.616667
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.40,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00,10.833333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3066761,2,2023-01-31 23:58:34,2023-02-01 00:12:33,NaN,3.05,NaN,None,107,48,0,15.80,0.00,0.5,3.96,0.0,1.0,23.76,NaN,NaN,13.983333
3066762,2,2023-01-31 23:31:09,2023-01-31 23:50:36,NaN,5.80,NaN,None,112,75,0,22.43,0.00,0.5,2.64,0.0,1.0,29.07,NaN,NaN,19.450000
3066763,2,2023-01-31 23:01:05,2023-01-31 23:25:36,NaN,4.67,NaN,None,114,239,0,17.61,0.00,0.5,5.32,0.0,1.0,26.93,NaN,NaN,24.516667
3066764,2,2023-01-31 23:40:00,2023-01-31 23:53:00,NaN,3.15,NaN,None,230,79,0,18.15,0.00,0.5,4.43,0.0,1.0,26.58,NaN,NaN,13.000000


### Question 4

In [9]:
df_Jan_subset['DOID'] = df_Jan_subset['DOLocationID'].astype(str)
df_Jan_subset['PUID'] = df_Jan_subset['PULocationID'].astype(str)
df_Jan_one = df_Jan_subset.loc[:, ['DOID', 'PUID']]
X_train = pd.get_dummies(df_Jan_one, columns = ['DOID', 'PUID'])
print(f"The new matrix has {X_train.shape[1]} columns.")

The new matrix has 515 columns.


### Question 5